In [1]:
import pandas as pd
import fiona
import shapely
import geopandas as gpd
import shapely.speedups
from shapely.geometry import shape, Point
import numpy as np
shapely.speedups.enable()

In [18]:
zones = fiona.open("../preprocessed_data/zones/fixed_taxi_zones.shp")
crash_df = pd.read_csv("../raw_data/car_crashes.csv")

#Filter valid data
crash_df = crash_df[crash_df['LATITUDE'].notna()]
crash_df = crash_df[crash_df['LONGITUDE'].notna()]

crash_df["CRASH DATE"] = crash_df["CRASH DATE"].astype("datetime64[ns]")
crash_df['YEAR'] = crash_df['CRASH DATE'].dt.year
crash_df['MONTH'] = crash_df['CRASH DATE'].dt.month
crash_df = crash_df[(crash_df['YEAR'] == 2018) | (crash_df['YEAR'] == 2019)]
crash_df = crash_df[crash_df['MONTH'] < 7]

crash_df = crash_df.drop(['BOROUGH', 'ZIP CODE', 'ON STREET NAME', 'CROSS STREET NAME', \
                          'OFF STREET NAME', 'NUMBER OF PERSONS INJURED', \
                          'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED', \
                          'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED', \
                          'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED', \
                          'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1', \
                          'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3', \
                          'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5', \
                          'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', \
                          'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5'], axis = 1)

/opt/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [24]:
#Finds which taxi zone a long/lat point is in
def find_zone(x,y):
    for i in range(len(zones)):
        shapefile_record = zones[i]
        zone_id = shapefile_record['properties']['LocationID']

    # Use Shapely to create the polygon
        shape = shapely.geometry.asShape(shapefile_record['geometry'])
        
        try:
            checkpoint = shapely.geometry.Point(float(x), float(y)) # longitude, latitude
        except:
            return 0

        if shape.contains(checkpoint):
            return zone_id
    return 0

In [21]:
#This takes a very long time to run (>8 hrs)
#Compute which taxi zone each crash is in
crash_df['zone'] = np.vectorize(find_zone)(crash_df["LONGITUDE"], crash_df["LATITUDE"])

In [1]:
crash_df = crash_df[crash_df['zone'] != 0]

NameError: name 'crash_df' is not defined

In [ ]:
crash_df.to_pickle("../preprocessed_data/crashes_w_zones.pkl")